In [16]:
import pandas as pd
import os
import json
import numpy as np
from os.path import dirname

root_path = dirname(os.getcwd())

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/comuzzi/_processed/"
data_dir_graphs = root_path + "/data/datasets/comuzzi/graphs_repair/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/comuzzi/_processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/comuzzi/graphs_repair/


In [ ]:
ACT_TIME_ONLY = True

In [17]:
ACT_TIME_ONLY = False

In [18]:
with open("dataset_features.json", 'r') as file:
    datasets_info = json.load(file)


In [ ]:
list(datasets_info.keys())

In [19]:
dataset = "BPI20_RequestForPayment_CZ"

In [20]:
nan_methods = ["odd", "even", "random", "window"]

masked_datasets = {key : pd.read_csv(f"{data_dir_processed}/{dataset}/{dataset}_masked_{key}_all.csv") for key in nan_methods}


In [21]:
tab_all = pd.read_csv(f"{data_dir_processed}/{dataset}/{dataset}_processed_all.csv") 
tab_all.head()

,CaseID,Activity,time:timestamp,org:resource,org:role,case:Project,case:Task,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,1,Request For Payment SUBMITTED by EMPLOYEE,0.000000,STAFF MEMBER,EMPLOYEE,project 148216,UNKNOWN,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
1,1,Request For Payment FINAL_APPROVED by SUPERVISOR,3.737670,STAFF MEMBER,SUPERVISOR,project 148216,UNKNOWN,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
2,1,Request For Payment REJECTED by MISSING,11.499982,STAFF MEMBER,MISSING,project 148216,UNKNOWN,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
3,1,Request For Payment SUBMITTED by EMPLOYEE,15.337479,STAFF MEMBER,EMPLOYEE,project 148216,UNKNOWN,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215
4,1,Request For Payment APPROVED by PRE_APPROVER,15.337486,STAFF MEMBER,PRE_APPROVER,project 148216,UNKNOWN,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215


In [22]:
tab_train = pd.read_csv(f"{data_dir_processed}/{dataset}/{dataset}_processed_train.csv")
tab_valid = pd.read_csv(f"{data_dir_processed}/{dataset}/{dataset}_processed_valid.csv")
tab_test = pd.read_csv(f"{data_dir_processed}/{dataset}/{dataset}_processed_test.csv")

In [ ]:
if dataset == "BPI_Challenge_2012_W_Complete":
    tab_all["org:resource"] = tab_all["org:resource"].astype(np.str_)
    tab_train["org:resource"] = tab_train["org:resource"].astype(np.str_)
    tab_valid["org:resource"] = tab_valid["org:resource"].astype(np.str_)
    tab_test["org:resource"] = tab_test["org:resource"].astype(np.str_)

In [23]:
with open("dataset_features.json", 'r') as file:
    dataset_info = json.load(file)[dataset]

In [ ]:
dataset_info

In [24]:
if ACT_TIME_ONLY:
    categorical_columns = ["Activity"]
    real_value_columns = ["time:timestamp"]
    dataset = f"{dataset}_AT_only"
else:
    categorical_columns = dataset_info["categorical"]
    real_value_columns = dataset_info["numerical"]

In [25]:
for k in categorical_columns:
    tab_all[k] = tab_all[k].astype("object")
    tab_train[k] = tab_train[k].astype("object")
    tab_valid[k] = tab_valid[k].astype("object")
    tab_test[k] = tab_test[k].astype("object")
    
    for k_m in masked_datasets:
        masked_datasets[k_m][k] = masked_datasets[k_m][k].astype("object")

In [ ]:
from numpy import NaN
if dataset == "sp2020_CZ":
    tab_all["REPAIR_IN_TIME_5D"] = [float(x) if x is not NaN else x for x in tab_all["REPAIR_IN_TIME_5D"].values]
    tab_train["REPAIR_IN_TIME_5D"] = [float(x) if x is not NaN else x for x in tab_train["REPAIR_IN_TIME_5D"].values]
    tab_valid["REPAIR_IN_TIME_5D"] = [float(x) if x is not NaN else x for x in tab_valid["REPAIR_IN_TIME_5D"].values]
    tab_test["REPAIR_IN_TIME_5D"] = [float(x) if x is not NaN else x for x in tab_test["REPAIR_IN_TIME_5D"].values]
    for k_m in masked_datasets:
        masked_datasets[k_m]["REPAIR_IN_TIME_5D"] = [float(x) if x is not NaN else x for x in masked_datasets[k_m]["REPAIR_IN_TIME_5D"].values]

In [ ]:
for k in categorical_columns:
    print(f"{k} {tab_test[k].values.dtype}")

In [ ]:
dataset

In [37]:
from numpy import NaN
from math import log
if dataset == "BPI20_RequestForPayment_CZ":
    tab_all["case:RequestedAmount"] = [log(x) if x > 0 else 0. for x in tab_all["case:RequestedAmount"].values]
    tab_train["case:RequestedAmount"] = [log(x) if x > 0 else 0. for x in tab_train["case:RequestedAmount"].values]
    tab_valid["case:RequestedAmount"] = [log(x) if x > 0 else 0. for x in tab_valid["case:RequestedAmount"].values]
    tab_test["case:RequestedAmount"] = [log(x) if x > 0 else 0. for x in tab_test["case:RequestedAmount"].values]
    for k_m in masked_datasets:
        masked_datasets[k_m]["case:RequestedAmount"] = [log(x) if x > 0 else 0. if x == 0 else x for x in masked_datasets[k_m]["case:RequestedAmount"].values]

In [38]:



masked_datasets[k_m]["case:RequestedAmount"]


0        3.536204
1             NaN
2             NaN
3        3.536204
4             NaN
           ...   
36791    2.734995
36792    2.734995
36793    2.734995
36794    2.734995
36795    2.734995
Name: case:RequestedAmount, Length: 36796, dtype: float64

In [ ]:
tab_all

### Prepare the graphs

In [39]:
from utils import get_case_ids, get_one_hot_encodings

from torch import tensor,int64, float32
from torch_geometric.data import HeteroData


In [40]:
MISSING_VALUE = "MISSING_VALUE"

In [41]:
import sklearn.preprocessing

def get_one_hot_encoder(dataset: pd.DataFrame, key: str):
    datas = np.append(dataset[key].unique(), MISSING_VALUE)
    datas = datas.astype(np.str_)
    datas = datas.reshape([len(datas), 1])
    onehot = sklearn.preprocessing.OneHotEncoder()
    onehot.fit(datas)
    return onehot

In [42]:
def get_node_features(dataset: pd.DataFrame, trace: pd.DataFrame, cat_features, real_features) -> dict:
    #columns_static = [c for c in trace if is_static(trace[c])]

    res = {}

    for key in trace:
        values = trace[key].values
        if key in cat_features:
            onehot_encoder = get_one_hot_encoder(dataset, key)
            values = values.astype(np.str_)
            #if key not in columns_static:
            try:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_encoder, values),
                    dtype=float32,
                    requires_grad=True
                )
            except ValueError:
                print(key)
                print(values)
            #else:
            #    res[key] = tensor(
            #        get_one_hot_encodings(onehot_encoder, np.array([values[0]])),
            #        dtype=float32,
            #        requires_grad=True
            #    )
        if key in real_features:
            #if key not in columns_static:
           
            res[key] = tensor(values,  dtype=float32,requires_grad=True)
            #else:
                #res[key] = tensor([values[0]], dtype=float32,requires_grad=True)
            res[key] = res[key].reshape(res[key].shape[0], 1)
        
    

    return res


In [43]:
def compute_edges_indexs(node_features: dict, prefix_len):
    res = {}
    keys = node_features.keys()
    # indexes = [[i, j] for i in range(prefix_len) for j in range(i + 1, prefix_len)]
    indexes = [[i, i + 1] for i in range(prefix_len-1)]
    # activities indexes
    for k in keys:
        if len(node_features[k]) != 1:
            if k == "Activity":
                res[(k, "followed_by", k)] = indexes
                for k2 in keys:
                    if k2 != k:
                        if len(node_features[k2]) == 1:
                            res[(k, "related_to", k2)] = [
                                [i, 0] for i in range(prefix_len)
                            ]
                        else:
                            res[(k, "related_to", k2)] = [
                                [i, i] for i in range(prefix_len)
                            ]
            else:
                res[(k, "related_to", k)] = indexes

    return res

In [ ]:
dataset

In [44]:
def get_masked_trace(dataset_traces, cat_features, real_features, caseid):
    trace = (
        dataset_traces.query(f"CaseID == '{caseid}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    
    if dataset != "bpi_2012_CZ" and dataset != 'bpi_2012_CZ_AT_only':
        mask = trace[trace.columns].isnull().apply(lambda x: all(x), axis=1)
    else:
        mask = trace[trace.columns].isnull().any(axis=1).values
        
    mask_index = [i for i in range(len(mask)) if mask[i]]
    
    for k in cat_features:
        for i in mask_index:
            trace.loc[i, k] = MISSING_VALUE
    
    for k in real_features:
        for i in mask_index:
            trace.loc[i, k] = -1
    
    return trace, mask
    
    

In [ ]:
nan_methods[0]

In [ ]:
masked_datasets[nan_methods[0]]

In [45]:

from copy import copy
import torch

def build_prefixes_graph_from_trace(dataset, trace, cat_features, real_features, caseid=None, mask_method=None):
    X = []  # graphs
   
    
    
    node_features = get_node_features(dataset, trace, cat_features, real_features)
    
    
    #masks = [f(trace) for f in mask_methods]
    
    if mask_method is not None:
        masked_methods = [mask_method]
    else:
        masked_methods = copy(nan_methods)
    
    
    for j in range(len(masked_methods)):
        
        G = HeteroData()
        
        #masked_trace, mask = modify_trace(trace, cat_features, real_features, masks[j])
        
        masked_trace, mask = get_masked_trace(masked_datasets[masked_methods[j]], cat_features, real_features, caseid)
        
        G.mask = torch.tensor(mask,dtype=torch.bool)
        
        
        node_features_masked_trace = get_node_features(dataset, masked_trace, cat_features, real_features)
        
        for k in node_features_masked_trace:
            G[k].x = node_features_masked_trace[k]


        edges_indexes = compute_edges_indexs(node_features_masked_trace, len(trace))

    


        for k in edges_indexes:
            ce = [[], []]
            for i in range(len(edges_indexes[k])):
                ce[0].append(edges_indexes[k][i][0])
                ce[1].append(edges_indexes[k][i][1])
            edges_indexes[k] = ce

        for k in edges_indexes:
            G[k].edge_index = tensor(edges_indexes[k], dtype=int64)

        G.y = {}
        for k in node_features:
            if k in cat_features:
                G.y[k] = torch.max(node_features[k], 1)[1]
            else:
                G.y[k] = node_features[k].reshape(1,-1)[0].detach().clone()
        
        
        
        X.append(G)
    
    

        
    return X

## Create the datasets

In [46]:
case_train_ids = get_case_ids(tab_train)
case_valid_ids = get_case_ids(tab_valid)
case_test_ids = get_case_ids(tab_test)

In [47]:
print(len(case_train_ids))
print(len(case_valid_ids))
print(len(case_test_ids))

4131
1377
1378


In [ ]:
trace2 = (
        masked_datasets["odd"].query(f"CaseID == '{case_train_ids[0]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
trace2

In [48]:
tab_train["CaseID"] = tab_train["CaseID"].astype(np.str_)
tab_valid["CaseID"] = tab_valid["CaseID"].astype(np.str_)
tab_test["CaseID"] = tab_test["CaseID"].astype(np.str_)

In [49]:
for k in masked_datasets:
    masked_datasets[k]["CaseID"] = masked_datasets[k]["CaseID"].astype(np.str_)

In [ ]:
tab_all["REPAIR_IN_TIME_5D"]

In [ ]:
masked_datasets["even"]["REPAIR_IN_TIME_5D"]

In [50]:
trace = (
        tab_train.query(f"CaseID == '{case_train_ids[0]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
trace 

,Activity,time:timestamp,org:resource,org:role,case:Project,case:Task,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,Request For Payment SUBMITTED by EMPLOYEE,0.000000,STAFF MEMBER,EMPLOYEE,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
1,Request For Payment FINAL_APPROVED by SUPERVISOR,3.737670,STAFF MEMBER,SUPERVISOR,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
2,Request For Payment REJECTED by MISSING,11.499982,STAFF MEMBER,MISSING,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
3,Request For Payment SUBMITTED by EMPLOYEE,15.337479,STAFF MEMBER,EMPLOYEE,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
4,Request For Payment APPROVED by PRE_APPROVER,15.337486,STAFF MEMBER,PRE_APPROVER,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
5,Request For Payment FINAL_APPROVED by SUPERVISOR,15.391828,STAFF MEMBER,SUPERVISOR,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
6,Request Payment,15.428686,SYSTEM,UNDEFINED,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215
7,Payment Handled,15.561581,SYSTEM,UNDEFINED,project 148216,UNKNOWN,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215


In [51]:
m_trace, mask = get_masked_trace(masked_datasets["random"], categorical_columns, real_value_columns, case_train_ids[0])

In [52]:
m_trace

,Activity,time:timestamp,org:resource,org:role,case:Project,case:Task,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber
0,Request For Payment SUBMITTED by EMPLOYEE,0.000000,STAFF MEMBER,EMPLOYEE,project 148216,UNKNOWN,organizational unit 65463,0.0,3.536204,UNKNOWN,request for payment number 148215
1,Request For Payment FINAL_APPROVED by SUPERVISOR,3.737670,STAFF MEMBER,SUPERVISOR,project 148216,UNKNOWN,organizational unit 65463,0.0,3.536204,UNKNOWN,request for payment number 148215
2,Request For Payment REJECTED by MISSING,11.499982,STAFF MEMBER,MISSING,project 148216,UNKNOWN,organizational unit 65463,0.0,3.536204,UNKNOWN,request for payment number 148215
3,MISSING_VALUE,-1.000000,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,NaN,-1.000000,MISSING_VALUE,MISSING_VALUE
4,MISSING_VALUE,-1.000000,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,NaN,-1.000000,MISSING_VALUE,MISSING_VALUE
5,Request For Payment FINAL_APPROVED by SUPERVISOR,15.391828,STAFF MEMBER,SUPERVISOR,project 148216,UNKNOWN,organizational unit 65463,0.0,3.536204,UNKNOWN,request for payment number 148215
6,MISSING_VALUE,-1.000000,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,NaN,-1.000000,MISSING_VALUE,MISSING_VALUE
7,MISSING_VALUE,-1.000000,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,MISSING_VALUE,NaN,-1.000000,MISSING_VALUE,MISSING_VALUE


In [ ]:
mask

In [53]:
graphs = build_prefixes_graph_from_trace(tab_all, trace, categorical_columns, real_value_columns, case_train_ids[0])

In [ ]:
graphs[0].x_dict["time:timestamp"]

In [ ]:
graphs[0].y

In [ ]:
graphs[0].mask

In [54]:
from tqdm.notebook import tqdm

In [55]:
print("Preparing training dataset...")

X_train = []


for i in tqdm(range(len(case_train_ids))):
    trace = (
        tab_train.query(f"CaseID == '{case_train_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )

    if len(trace) > 2:
        graphs = build_prefixes_graph_from_trace(
            dataset=tab_all,
            trace=trace,
            cat_features=categorical_columns,
            real_features=real_value_columns,
            caseid=case_train_ids[i]
        )
        for i in range(len(graphs)):
            X_train.append(graphs[i])


# Y_train = tensor(Y_train, dtype=float32)

print("Done!\n\n")

Preparing training dataset...


  0%|          | 0/4131 [00:00<?, ?it/s]

Done!




In [56]:
import pickle

In [57]:


with open(data_dir_graphs + dataset + "_TRAIN_repair.pkl", "wb") as f:
    pickle.dump(X_train, f)

In [58]:
del X_train

In [59]:
print("Preparing validation dataset...")

X_valid = []


for i in tqdm(range(len(case_valid_ids))):
    trace = (
        tab_valid.query(f"CaseID == '{case_valid_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    if len(trace) > 2:
        graphs = build_prefixes_graph_from_trace(
            dataset=tab_all,
            trace=trace,
            cat_features=categorical_columns,
            real_features=real_value_columns,
            caseid=case_valid_ids[i]
        )
        for i in range(len(graphs)):
            X_valid.append(graphs[i])


# Y_valid = tensor(Y_valid)

print("Done!\n\n")

Preparing validation dataset...


  0%|          | 0/1377 [00:00<?, ?it/s]

Done!




In [60]:
with open(data_dir_graphs + dataset + "_VALID_repair.pkl", "wb") as f:
    pickle.dump(X_valid, f)

In [61]:
del X_valid

In [62]:
print("Preparing test dataset...")

X_test = []


for i in tqdm(range(len(case_test_ids))):
    trace = (
        tab_test.query(f"CaseID == '{case_test_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )

    if len(trace) > 2:
        graphs = build_prefixes_graph_from_trace(
            dataset=tab_all,
            trace=trace,
            cat_features=categorical_columns,
            real_features=real_value_columns,
            caseid=case_test_ids[i]
        )
        for i in range(len(graphs)):
            X_test.append(graphs[i])


# Y_test = tensor(Y_test)

print("Done!\n\n")

Preparing test dataset...


  0%|          | 0/1378 [00:00<?, ?it/s]

Done!




In [63]:
with open(data_dir_graphs + dataset + "_TEST_repair.pkl", "wb") as f:
    pickle.dump(X_test, f)

In [64]:
del X_test

In [65]:
def create_and_save_test(case:str):
    X_test = []
    for i in tqdm(range(len(case_test_ids))):
        trace = (
            tab_test.query(f"CaseID == '{case_test_ids[i]}'")
            .reset_index()
            .drop(columns="index")
            .drop(columns="CaseID")
        )

        if len(trace) > 2:
            graphs = build_prefixes_graph_from_trace(
                dataset=tab_all,
                trace=trace,
                cat_features=categorical_columns,
                real_features=real_value_columns,
                caseid=case_test_ids[i],
                mask_method=case,
            )
            for i in range(len(graphs)):
                X_test.append(graphs[i])
    
    with open(data_dir_graphs + dataset + f"_TEST_repair_{case}.pkl", "wb") as f:
        pickle.dump(X_test, f)

In [66]:
create_and_save_test("even")

  0%|          | 0/1378 [00:00<?, ?it/s]

In [67]:
create_and_save_test("odd")

  0%|          | 0/1378 [00:00<?, ?it/s]

In [68]:
create_and_save_test("random")

  0%|          | 0/1378 [00:00<?, ?it/s]

In [69]:
create_and_save_test("window")

  0%|          | 0/1378 [00:00<?, ?it/s]